In [37]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/ubuntu/varios/skforecast'

In [38]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from skforecast.ForecasterAutoregMultiSeriesCustom import ForecasterAutoregMultiSeriesCustom

In [39]:
# Download data
# ==============================================================================
url = ('https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o.csv')
data = pd.read_csv(url, sep=',', header=0, names=['y', 'date'])
data['date'] = pd.to_datetime(data['date'], format='%Y/%m/%d')
data = data.set_index('date')
data = data.asfreq('MS')
data = data.y
data = pd.concat((data, data*10), axis=1)
data.columns = ['series_1', 'series_2']
exog_1 = pd.Series(np.arange(len(data)), index=data.index)
exog_2 = exog_1 * 10
exog = pd.concat((exog_1, exog_2), axis=1)
exog.columns = ['exog_1', 'exog_2']

data_train = data.loc[:'2007-12-31']
exog_train = exog.loc[:'2007-12-31']
data_test = data.loc['2008-01-01 00:00:00':]
exog_test = exog.loc['2008-01-01 00:00:00':]

In [40]:
# Custom function to create predictors
# ==============================================================================
def create_predictors(y):
    """
    Create first 3 lags of a time series.
    Calculate moving average with window 20.
    """

    lags = y[-1:-3:-1]
    mean = np.mean(y[-20:])
    predictors = np.hstack([lags, mean])

    return predictors

In [41]:
# Create forecaster
# ==============================================================================
forecaster = ForecasterAutoregMultiSeriesCustom(
                 regressor       = Ridge(random_state=123),
                 fun_predictors  = create_predictors,
                 window_size     = 20,
                 name_predictors = ['lag_1', 'lag_2', 'lag_3']
             )

In [42]:
X_train, y_train, y_index, y_train_index = forecaster.create_train_X_y(series=data, exog=exog)

In [43]:
forecaster.name_predictors

['lag_1', 'lag_2', 'lag_3']

In [44]:
X_train

,lag_1,lag_2,lag_3,exog_1,exog_2,series_1,series_2
0,0.387554,0.751503,0.496401,20.0,200.0,1.0,0.0
1,0.427283,0.387554,0.496275,21.0,210.0,1.0,0.0
2,0.413890,0.427283,0.496924,22.0,220.0,1.0,0.0
3,0.428859,0.413890,0.496759,23.0,230.0,1.0,0.0
4,0.470126,0.428859,0.495638,24.0,240.0,1.0,0.0
...,...,...,...,...,...,...,...
363,12.199410,11.765890,9.803897,199.0,1990.0,0.0,1.0
364,7.618220,12.199410,9.785823,200.0,2000.0,0.0,1.0
365,6.494350,7.618220,9.668384,201.0,2010.0,0.0,1.0
366,8.278870,6.494350,9.557504,202.0,2020.0,0.0,1.0


In [45]:
forecaster.name_predictors

['lag_1', 'lag_2', 'lag_3']

In [46]:
forecaster.fit(series=data_train, exog=exog_train)
forecaster

ForecasterAutoregMultiSeriesCustom 
Regressor: Ridge(random_state=123) 
Predictors created with function: create_predictors 
Transformer for series: None 
Transformer for exog: None 
Window size: 20 
Series levels (names): ['series_1', 'series_2'] 
Series weights: None 
Weight function included: False 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['exog_1', 'exog_2'] 
Training range: [Timestamp('1991-07-01 00:00:00'), Timestamp('2007-12-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: {'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'positive': False, 'random_state': 123, 'solver': 'auto', 'tol': 0.0001} 
Creation date: 2023-03-02 11:27:37 
Last fit date: 2023-03-02 11:27:38 
Skforecast version: 0.7.0 
Python version: 3.10.9 

In [47]:
forecaster.predict(levels='series_1', steps=3, exog=exog_test, last_window=data_train)

,series_1
2008-01-01,1.363601
2008-02-01,1.500655
2008-03-01,1.589148
